In [1]:
# coding: utf-8
#步骤一
import pandas as pd
import numpy as np
import os
import re
import warnings
warnings.filterwarnings('ignore')


#读取标签
df = pd.read_csv(r'\Users\Think\Desktop\testout.csv',index_col=0)
df = df.astype(np.int)
#文件读取
words = []
names = []
rootpath = r'\Users\Think\Desktop\stock_news'
for root, dirs, files in os.walk(rootpath, topdown=False):
    for name in files:
        filepath = os.path.join(root, name)
#         print(filepath)
        try:
            word = open(filepath,'r',encoding='utf8').read()
        except:
            continue
        words.append(word)
        names.append(name)
df_word = pd.DataFrame()
df_word['word'] = words
df_word['name'] = names


#标注数据
def get(x):
    date = int(x[:8])
    stock_code = x[x.index('-')+1:x.index('.')]
    stock_code = stock_code[:6]+'.'+stock_code[6:]
    try:
        a = df.loc[date+1,stock_code]
    except:
        a = ''
    return a   
df_word['label'] = df_word['name'].apply(lambda x:get(x))

#print(df_word)

#标注数据
def getcode(x):
    stock_code = x[x.index('-')+1:x.index('.')]
    stock_code = stock_code[:6]+'.'+stock_code[6:]
    return stock_code
df_word['bank'] = df_word['name'].apply(lambda x:getcode(x))

df_final = df_word[df_word['label'] != '']

#查看数据
df_final.head(10)
#查看涨跌总数
#df_final.groupby('label').count()

#保存标注好的数据
#df_final.to_excel(r'\Users\Think\Desktop\df_final_test.xlsx')

,word,name,label,bank
8292,年 一年 A股 市场 解禁 上市 亿股 市值 约 万亿元 占 A股 总 股本 总 市值 比例...,20190101051527-601998SH.txt,0,601998.SH
8293,年 月 日 河北 证监局 网站 公布 行政处罚 决定书 号 显示 周 永平 买卖 顺网 科技...,20190101080010-601998SH.txt,0,601998.SH
8294,年 月 日 吉林 证监局 网站 发布 行政处罚 决定书 号 显示 SZ 假央企 事件 信披 ...,20190101080026-601998SH.txt,0,601998.SH
8295,年 第一个 月 解禁 市值 超 亿元 A股 承压 年首 月 A股 市场 迎 大规模 解禁 潮...,20190101111458-601998SH.txt,0,601998.SH
8296,月份 面临 解禁 个股 数量 年 单月 最新 收盘价 计算 合计 解禁 市值 超 亿元 股 ...,20190101130802-601998SH.txt,0,601998.SH
8297,上周 债市 上涨 国债 平均 下行 7bp AAA 级 AA 级 企业债 城 投债 下行 4...,20190101140319-601998SH.txt,0,601998.SH
8298,去年 月 日 标普 道琼斯 指数 年 月 日 正式 纳入 A股 美国 时间 月 日 标普 道...,20190101141543-601998SH.txt,0,601998.SH
8299,年 银行业 可谓 跌宕起伏 年 严 监管 年 执行 贯彻 一年 年 银保监 系统对 银行业 ...,20190101203453-601998SH.txt,0,601998.SH
8300,年 总算 年 交易日 美国 三大 股指 集体 收涨 道 指 纳指 标普 指数 下跌 录得 年...,20190101204027-601998SH.txt,0,601998.SH
8301,多年 发展 壹玖壹 玖 酒类 平台 科技股份 有限公司 简称 酒类 行业 积累 经验 酒类 ...,20190102004033-601998SH.txt,1,601998.SH


In [3]:
#步骤二：文本向量化，词典模型
import keras
from keras.preprocessing import text as T
vectorizer = T.Tokenizer(lower=False,split=
' ',num_words=None,char_level=False,filters='')
document_after = df_final['word'].values
vectorizer.fit_on_texts(document_after)
text_sequences = vectorizer.texts_to_sequences(document_after)


In [4]:
#加入新的文本
df_test = pd.read_excel(r'\Users\Think\Desktop\df_final_test.xlsx')
df_test['word'] = df_test.fillna({'word':''})
text1 = df_test['word'].values
vectorizer.word_counts
text_sequences = vectorizer.texts_to_sequences(text1)

from keras.preprocessing import sequence
max_len = 1000
text_sequences_padded = sequence.pad_sequences(text_sequences, maxlen=max_len)


#查看内容及类型
#print(text_sequences_padded.shape)
#type(text_sequences_padded)

#保存文本表示，array类型转dataframe
pd.DataFrame(text_sequences_padded).to_csv(r'\Users\Think\Desktop\text_sequences_padded_test.csv')

In [6]:
'''步骤四：导入模型工具包'''
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,SpatialDropout1D
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D, Flatten, LSTM
from keras.datasets import imdb
from keras.utils import to_categorical
from keras import metrics

In [5]:
#定义测试集数据
X_test = pd.DataFrame(text_sequences_padded)
print(X_test.head(), X_test.shape)
Y_test = pd.get_dummies(df_test['label'])

    0    1      2     3    4      5     6    7    8    9    ...      990  \
0     0    0      0     0    0      0     0    0    0    0  ...     1288   
1  1497   60  12632  1497   60  37980  8496   66  816  371  ...    10014   
2     0    0      0     0    0      0     0    0    0    0  ...      255   
3     0    0      0     0    0      0     0    0    0    0  ...      923   
4     0    0      0     0    0      0     0    0    0    0  ...      346   

    991   992   993   994   995   996   997    998    999  
0  7608   141   355    94    62    52    47   2784    813  
1  8119  1296   342  2333   255    52    47      8   2163  
2  3967  2281     1     3     6    52    47      8   2163  
3   439  3492   180   157  1288  2148  1014  13931  13932  
4     4    56  1288   157   346     4    52     47    869  

[5 rows x 1000 columns] (6271, 1000)


In [7]:
'''步骤五：直接导入模型并预测'''
import numpy as np
from keras.models import load_model
model1 = load_model(r'\Users\Think\Desktop\my_model_final2.h5')

model1.summary()
Y_pred = np.array(model1.predict(X_test))
Y_class = np.array(model1.predict_classes(X_test))
print(Y_pred,Y_class)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1000, 200)         38583000  
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 1000, 200)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1000, 250)         150250    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 250)               0         
_________________________________________________________________
dense_4 (Dense)            

In [8]:

model1.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
score = model1.evaluate(X_test, Y_test, batch_size=32)
print(score)

6271/6271 [==============================] - 24s 4ms/step
[0.8757294074444656, 0.5157072236522345]
